In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# read in file
df = pd.read_csv('dwellings_ml.txt')

__Highlight the Questions and Tasks__

## QUESTION|TASK 1

__Create 2-3 charts that evaluate potential relationships between the home variables and before1980. Explain what you learn from the charts that could help a machine learning algorithm.__

_In the chart down below, outliers are severely affecting and shifting the data. Although, one thing that I noticed in this chart is that although there are many outliers for before and after 1980, there is a significant increase in sales price after the year 1980. However, after 1980 has the highest outlier close to $10M._


In [ ]:
# convert before1980 to a categorical variable
df['before1980'] = df['before1980'].astype('category')

# shart for sales price comparison for before1980 status
chart1 = px.box(df, x='before1980', 
y='sprice', 
color='before1980', 
title='Sales Price by Before1980 Status')

# change y axis range
chart1.update_yaxes(range=[0, 10000000])

# change axis titles
chart1.update_layout(
    xaxis_title='Before1980 Status',
    yaxis_title='Sales Price')

chart1.show()

_The chart below shows the relationship between the number homes built each year as time goes on before and after 1980. I noticed that before 1980 the number of houses built was never consistent. It seemed to fluctuate a lot goin up and down on how many houses were built within the year. After 1980 the number of houses built dropped around the 90s and then consistently increased going into the 2000s higher than how many houses were built before the 1980s._


In [ ]:
# chart for number of homes built before and after 1980
chart2 = px.histogram(df, x='yrbuilt', 
color='before1980', 
title='Year Built Distribution by Before1980 Status')

#change axis titles
chart2.update_layout(
  xaxis_title='Year Built',
  yaxis_title='Number of Homes Built'
)

# make outside line of histogram bars black
chart2.update_traces(marker_line_color='black', marker_line_width=2)

chart2.show()

_Below is a chart that shows the relationship of how many bedrooms a house had depending on the sales price of the home. Most of the houses either before or after 1980 had outliers that affect and shift the data. Although, the houses built after 1980 had not only more bedrooms, but they had more outliers in sales price. This could have been caused by the great recession of 2008._


In [ ]:
# chart for number of bedrooms before and after 1980
chart3 = px.box(df, x='numbdrm', 
y='sprice', 
color='before1980', 
title='Number of Bedrooms vs. Sale Price by Before1980 Status')

# change axis titles
chart3.update_layout(
  xaxis_title='Number of Bedrooms', 
  yaxis_title='Sales Price'
)

# change the interval scale of the x axis
chart3.update_xaxes(dtick=1)

chart3.show()

## QUESTION|TASK 2

__Build a classification model labeling houses as being built “before 1980” or “during or after 1980”. Your goal is to reach or exceed 90% accuracy. Explain your final model choice (algorithm, tuning parameters, etc) and describe what other models you tried.__

_I did a model of Logistic Regression and a model for Random Forest Classifier. Both models are very accurate and were not off by that much accuracy from each other. Although, the Random Forest Classifier model had a perfect accuracy so that one would be the model to choose._


In [ ]:
# split the data
df_drop_parcel = df.drop(['before1980', 'parcel'], axis=1)
df_no_drop_parcel = df['before1980']

# Splitting the dataset into training and testing sets again
df_drop_parcel_train, df_drop_parcel_test, df_no_drop_parcel_train, df_no_drop_parcel_test = train_test_split(df_drop_parcel, df_no_drop_parcel, test_size=0.2, random_state=42)

# Standardizing the features again
scaler = StandardScaler()
df_drop_parcel_train_scaled = scaler.fit_transform(df_drop_parcel_train)
df_drop_parcel_test_scaled = scaler.transform(df_drop_parcel_test)

# Logistic Regression Model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(df_drop_parcel_train_scaled, df_no_drop_parcel_train)
df_no_drop_parcel_pred_log_reg = log_reg.predict(df_drop_parcel_test_scaled)
accuracy_log_reg = accuracy_score(df_no_drop_parcel_test, df_no_drop_parcel_pred_log_reg)

# Random Forest Classifier
rf_clf = RandomForestClassifier(random_state=42) # Side Note: AI was used to come up with and format lines 147-150 of this code for this question
rf_clf.fit(df_drop_parcel_train_scaled, df_no_drop_parcel_train)
df_no_drop_parcel_pred_rf_clf = rf_clf.predict(df_drop_parcel_test_scaled)
accuracy_rf_clf = accuracy_score(df_no_drop_parcel_test, df_no_drop_parcel_pred_rf_clf)

print('Logistic Regression Accuracy:', accuracy_log_reg)
print('Random Forest Classifier Accuracy:', accuracy_rf_clf)

## QUESTION|TASK 3

__Justify your classification model by discussing the most important features selected by your model. This discussion should include a chart and a description of the features.__


In [ ]:
# get feature importance
feature_importances = rf_clf.feature_importances_

# create a feature dataframe
features_df = pd.DataFrame({'Feature': df_drop_parcel.columns, 'Importance': feature_importances})
features_df = features_df.sort_values(by='Importance', ascending=False)

# remove yrbuilt column
features_df = features_df[features_df['Feature'] != 'yrbuilt']

_While making this chart the column 'yrbuilt' was an extreme outlier compared to the rest of the features for a house so that column was removed so there would be a more accurate comparison for the rest of the features. The top 3 features for a house are the number of stories, the number of bathrooms, and the quality of the home. After those 3 most of the data is around the same or not as important for the ratio of the chart. This means that houses that have more features better for the ones of more importance are likely to sell better and faster._


In [ ]:
# make a chart
chart4 = px.bar(features_df, x='Importance', y='Feature', title='Feature Importance in Random Forest Classifier', orientation='h')
chart4.update_layout(yaxis={'categoryorder':'total ascending'})

# change axis titles
chart4.update_layout(
  xaxis_title='Importance', 
  yaxis_title='Feature'
)

chart4.show()

## QUESTION|TASK 4

__Describe the quality of your classification model using 2-3 different evaluation metrics. You also need to explain how to interpret each of the evaluation metrics you use.__

_type your results and analysis here_
